<a href="https://colab.research.google.com/github/dhahbimohamed/PC-Parts-Recommender-System/blob/main/Notebooks/ncf_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load 'Cleaned_GPU_Data.csv'
df_gpus = pd.read_csv('/content/Cleaned_GPU_Data .csv')
print("\n--- df_gpus Head ---")
print(df_gpus.head())
print("\n--- df_gpus Info ---")
df_gpus.info()
print("\n--- df_gpus Describe ---")
print(df_gpus.describe())

# Load 'clean_motherbords_Data.csv'
df_motherboards = pd.read_csv('/content/clean_motherbords_Data.csv')
print("\n--- df_motherboards Head ---")
print(df_motherboards.head())
print("\n--- df_motherboards Info ---")
df_motherboards.info()
print("\n--- df_motherboards Describe ---")
print(df_motherboards.describe())

# Load 'clean_data_RAM.csv'
df_ram = pd.read_csv('/content/clean_data_RAM.csv')
print("\n--- df_ram Head ---")
print(df_ram.head())
print("\n--- df_ram Info ---")
df_ram.info()
print("\n--- df_ram Describe ---")
print(df_ram.describe())

# Load 'cleaned_cpus.csv'
df_cpus = pd.read_csv('/content/cleaned_cpus.csv')
print("\n--- df_cpus Head ---")
print(df_cpus.head())
print("\n--- df_cpus Info ---")
df_cpus.info()
print("\n--- df_cpus Describe ---")
print(df_cpus.describe())


--- df_gpus Head ---
                                                Name GPU_Brand  \
0                   Acer Nitro OC Arc B58036th / 101     Intel   
1          Acer Nitro OC Radeon RX 7700 XT28th / 101       AMD   
2  Acer Predator BiFrost Radeon RX 7800 XT OC21st...       AMD   
3       ASRock Arc B580 Challenger 12GB OC36th / 101     Intel   
4       ASRock Arc B580 Challenger 12GB OC36th / 101     Intel   

         GPU_Chipset                         Model  Memory_Size Memory_Type  \
0           Arc B580       DP.Z4BWW.P01AN-B580-OCA         12.0       GDDR6   
1  Radeon RX 7700 XT   DP.Z39WW.P01AN-RX7700XT-OCB         12.0       GDDR6   
2  Radeon RX 7800 XT    DP.Z3AWW.P01PB-RX7800XT-OC         16.0       GDDR6   
3           Arc B580  B580 CL 12GO90-GA5LZZ-00UANF         12.0       GDDR6   
4           Arc B580  B580 CL 12GO90-GA5LZZ-00UANF         12.0       GDDR6   

   Base_Clock  Boost_Clock    TDP Interface  ...  Power_Connectors  \
0      2670.0       2740.0  190.0  P

## Merge and Standardize Component Data


In [ ]:
print("Dropping 'Audio Channels' from df_motherboards...")
df_motherboards = df_motherboards.drop(columns=['Audio Channels'])
print("'Audio Channels' dropped from df_motherboards.")

print("Dropping 'Unnamed: 0' from df_ram...")
df_ram = df_ram.drop(columns=['Unnamed: 0'])
print("'Unnamed: 0' dropped from df_ram.")

Dropping 'Audio Channels' from df_motherboards...
'Audio Channels' dropped from df_motherboards.
Dropping 'Unnamed: 0' from df_ram...
'Unnamed: 0' dropped from df_ram.


In [ ]:
print("Correcting 'Speed' in df_ram...")
# Assume speeds greater than a reasonable threshold (e.g., 10000 MHz) are likely in 10x units (e.g., 56000 should be 5600)
df_ram.loc[df_ram['Speed'] > 10000, 'Speed'] = df_ram.loc[df_ram['Speed'] > 10000, 'Speed'] / 10
print("'Speed' in df_ram corrected. Min: {:.2f}, Max: {:.2f}".format(df_ram['Speed'].min(), df_ram['Speed'].max()))

print("Correcting 'Cores', 'Threads', 'Base Clock', 'Turbo Clock' in df_cpus...")
# Cap extreme outliers in 'Cores' and 'Threads' to a more realistic maximum (e.g., 64 cores/128 threads for consumer/HEDT CPUs)
# For Cores, we observed a max of 8.06e+12, let's cap it at 64 as a reasonable upper bound for consumer CPUs.
# For Threads, we observed a max of 192, which is high but plausible for some server CPUs, but given the consumer context, capping it might be necessary. Let's review first.
# Re-inspecting the describe output for Cores, it's clear the mean/std are skewed by extreme outliers, while 75th percentile is 16.
# Let's cap Cores at 64 and Threads at 128 as a safe upper bound.
df_cpus['Cores'] = df_cpus['Cores'].apply(lambda x: x if x < 100 else df_cpus['Cores'].quantile(0.75))
df_cpus['Threads'] = df_cpus['Threads'].apply(lambda x: x if x < 200 else df_cpus['Threads'].quantile(0.75))

# Correct 'Base Clock' and 'Turbo Clock' values: if a value is very low (e.g., < 100), it's likely in GHz and needs to be converted to MHz
df_cpus.loc[df_cpus['Base Clock'] < 100, 'Base Clock'] = df_cpus.loc[df_cpus['Base Clock'] < 100, 'Base Clock'] * 1000
df_cpus.loc[df_cpus['Turbo Clock'] < 100, 'Turbo Clock'] = df_cpus.loc[df_cpus['Turbo Clock'] < 100, 'Turbo Clock'] * 1000
print("'Cores', 'Threads', 'Base Clock', 'Turbo Clock' in df_cpus corrected.")

print("Handling missing values in 'Integrated Graphics' in df_cpus...")
df_cpus['Integrated Graphics'] = df_cpus['Integrated Graphics'].fillna('No Integrated Graphics')
print("'Integrated Graphics' missing values handled.")

# Display updated descriptive statistics for these columns to verify
print("\n--- df_ram['Speed'] Describe after correction ---")
print(df_ram['Speed'].describe())
print("\n--- df_cpus numeric columns Describe after correction ---")
print(df_cpus[['Cores', 'Threads', 'Base Clock', 'Turbo Clock', 'Integrated Graphics']].describe(include='all'))

Correcting 'Speed' in df_ram...
'Speed' in df_ram corrected. Min: 3200.00, Max: 7200.00
Correcting 'Cores', 'Threads', 'Base Clock', 'Turbo Clock' in df_cpus...
'Cores', 'Threads', 'Base Clock', 'Turbo Clock' in df_cpus corrected.
Handling missing values in 'Integrated Graphics' in df_cpus...
'Integrated Graphics' missing values handled.

--- df_ram['Speed'] Describe after correction ---
count     464.000000
mean     5853.793103
std       476.659824
min      3200.000000
25%      5590.000000
50%      5680.000000
75%      6000.000000
max      7200.000000
Name: Speed, dtype: float64

--- df_cpus numeric columns Describe after correction ---
             Cores     Threads   Base Clock   Turbo Clock  \
count   828.000000  828.000000   828.000000    828.000000   
unique         NaN         NaN          NaN           NaN   
top            NaN         NaN          NaN           NaN   
freq           NaN         NaN          NaN           NaN   
mean     12.314010   21.369565  3804.589372   478

In [ ]:
print("Adding 'component_type' and 'component_id' to df_gpus...")
df_gpus['component_type'] = 'GPU'
df_gpus['component_id'] = 'GPU_' + df_gpus.index.astype(str)
print("'component_type' and 'component_id' added to df_gpus.")

print("Adding 'component_type' and 'component_id' to df_motherboards...")
df_motherboards['component_type'] = 'Motherboard'
df_motherboards['component_id'] = 'MB_' + df_motherboards.index.astype(str)
print("'component_type' and 'component_id' added to df_motherboards.")

print("Adding 'component_type' and 'component_id' to df_ram...")
df_ram['component_type'] = 'RAM'
df_ram['component_id'] = 'RAM_' + df_ram.index.astype(str)
print("'component_type' and 'component_id' added to df_ram.")

print("Adding 'component_type' and 'component_id' to df_cpus...")
df_cpus['component_type'] = 'CPU'
df_cpus['component_id'] = 'CPU_' + df_cpus.index.astype(str)
print("'component_type' and 'component_id' added to df_cpus.")

# Display head of each DataFrame with new columns to verify
print("\n--- df_gpus with new columns ---")
print(df_gpus[['Name', 'component_type', 'component_id']].head())
print("\n--- df_motherboards with new columns ---")
print(df_motherboards[['Name', 'component_type', 'component_id']].head())
print("\n--- df_ram with new columns ---")
print(df_ram[['Name', 'component_type', 'component_id']].head())
print("\n--- df_cpus with new columns ---")
print(df_cpus[['Name', 'component_type', 'component_id']].head())

Adding 'component_type' and 'component_id' to df_gpus...
'component_type' and 'component_id' added to df_gpus.
Adding 'component_type' and 'component_id' to df_motherboards...
'component_type' and 'component_id' added to df_motherboards.
Adding 'component_type' and 'component_id' to df_ram...
'component_type' and 'component_id' added to df_ram.
Adding 'component_type' and 'component_id' to df_cpus...
'component_type' and 'component_id' added to df_cpus.

--- df_gpus with new columns ---
                                                Name component_type  \
0                   Acer Nitro OC Arc B58036th / 101            GPU   
1          Acer Nitro OC Radeon RX 7700 XT28th / 101            GPU   
2  Acer Predator BiFrost Radeon RX 7800 XT OC21st...            GPU   
3       ASRock Arc B580 Challenger 12GB OC36th / 101            GPU   
4       ASRock Arc B580 Challenger 12GB OC36th / 101            GPU   

  component_id  
0        GPU_0  
1        GPU_1  
2        GPU_2  
3        GPU_

In [ ]:
print("Standardizing column names across DataFrames...")

# Standardize 'Name' to 'Product_Name' for all DataFrames
df_gpus = df_gpus.rename(columns={'Name': 'Product_Name'})
df_motherboards = df_motherboards.rename(columns={'Name': 'Product_Name'})
df_ram = df_ram.rename(columns={'Name': 'Product_Name'})
df_cpus = df_cpus.rename(columns={'Name': 'Product_Name'})

# Standardize 'Brand' to 'Manufacturer'
df_gpus = df_gpus.rename(columns={'GPU_Brand': 'Manufacturer'})
df_motherboards = df_motherboards.rename(columns={'Brand': 'Manufacturer'})
df_ram = df_ram.rename(columns={'Brand': 'Manufacturer'})
df_cpus = df_cpus.rename(columns={'Brand': 'Manufacturer'})

# Rename 'Type' in df_ram to 'Memory_Type' for consistency with df_gpus
df_ram = df_ram.rename(columns={'Type': 'Memory_Type'})

# Rename 'Socket' in df_cpus and df_motherboards to 'CPU_Socket'
df_cpus = df_cpus.rename(columns={'Socket': 'CPU_Socket'})
df_motherboards = df_motherboards.rename(columns={'Socket': 'CPU_Socket'})

# Rename 'Interface' in df_gpus to 'PCIe_Interface'
df_gpus = df_gpus.rename(columns={'Interface': 'PCIe_Interface'})

# Rename 'TDP' in df_gpus and df_cpus to 'Thermal_Design_Power'
df_gpus = df_gpus.rename(columns={'TDP': 'Thermal_Design_Power'})
df_cpus = df_cpus.rename(columns={'TDP': 'Thermal_Design_Power'})


# Display some column names to verify
print("df_gpus columns after standardization:", df_gpus.columns.tolist()[:5])
print("df_motherboards columns after standardization:", df_motherboards.columns.tolist()[:5])
print("df_ram columns after standardization:", df_ram.columns.tolist()[:5])
print("df_cpus columns after standardization:", df_cpus.columns.tolist()[:5])
print("Column names standardized.")

Standardizing column names across DataFrames...
df_gpus columns after standardization: ['Product_Name', 'Manufacturer', 'GPU_Chipset', 'Model', 'Memory_Size']
df_motherboards columns after standardization: ['URL', 'Seller_Product_URL', 'Availability', 'Price', 'Product_Name']
df_ram columns after standardization: ['URL', 'Seller_Product_URL', 'Price', 'Product_Name', 'Manufacturer']
df_cpus columns after standardization: ['Product_Name', 'Manufacturer', 'Model', 'Cores', 'Threads']
Column names standardized.


In [ ]:
print("Selecting relevant columns from each DataFrame...")

# Columns for GPUs
gpu_cols = [
    'component_id', 'Product_Name', 'component_type', 'Manufacturer', 'GPU_Chipset',
    'Memory_Size', 'Memory_Type', 'PCIe_Interface', 'Thermal_Design_Power',
    'SLI_Support', 'CrossFire_Support', 'Price'
]
df_gpus_selected = df_gpus[gpu_cols]

# Columns for Motherboards
mb_cols = [
    'component_id', 'Product_Name', 'component_type', 'Manufacturer', 'CPU_Socket',
    'Chipset', 'Form Factor', 'RAM Slots', 'Max RAM', 'ECC Support',
    'PCIe x16 Slots', 'M.2 Ports', 'SATA Ports', 'Wireless LAN', 'Wi-Fi Generation',
    'SLI Support', 'CrossFire Support', 'Price'
]
df_motherboards_selected = df_motherboards[mb_cols]

# Columns for RAM
ram_cols = [
    'component_id', 'Product_Name', 'component_type', 'Manufacturer', 'Memory_Type',
    'Size', 'Speed', 'CAS Latency', 'Voltage', 'Price'
]
df_ram_selected = df_ram[ram_cols]

# Columns for CPUs
cpu_cols = [
    'component_id', 'Product_Name', 'component_type', 'Manufacturer', 'Model',
    'Cores', 'Threads', 'Base Clock', 'Turbo Clock', 'CPU_Socket',
    'Thermal_Design_Power', 'L3 Cache', 'Process', 'Integrated Graphics', 'Price'
]
df_cpus_selected = df_cpus[cpu_cols]

print("Relevant columns selected.")

# Display the shape of the selected DataFrames to confirm
print("\n--- Shapes of Selected DataFrames ---")
print(f"df_gpus_selected: {df_gpus_selected.shape}")
print(f"df_motherboards_selected: {df_motherboards_selected.shape}")
print(f"df_ram_selected: {df_ram_selected.shape}")
print(f"df_cpus_selected: {df_cpus_selected.shape}")


Selecting relevant columns from each DataFrame...
Relevant columns selected.

--- Shapes of Selected DataFrames ---
df_gpus_selected: (1184, 12)
df_motherboards_selected: (739, 18)
df_ram_selected: (464, 10)
df_cpus_selected: (828, 15)


In [ ]:
print("Concatenating all selected DataFrames into df_components...")
df_components = pd.concat([
    df_gpus_selected,
    df_motherboards_selected,
    df_ram_selected,
    df_cpus_selected
], ignore_index=True)

print("Concatenation complete. Displaying head, info, and describe of df_components...")

print("\n--- df_components Head ---")
print(df_components.head())

print("\n--- df_components Info ---")
df_components.info()

print("\n--- df_components Describe ---")
print(df_components.describe(include='all'))

Concatenating all selected DataFrames into df_components...
Concatenation complete. Displaying head, info, and describe of df_components...

--- df_components Head ---
  component_id                                       Product_Name  \
0        GPU_0                   Acer Nitro OC Arc B58036th / 101   
1        GPU_1          Acer Nitro OC Radeon RX 7700 XT28th / 101   
2        GPU_2  Acer Predator BiFrost Radeon RX 7800 XT OC21st...   
3        GPU_3       ASRock Arc B580 Challenger 12GB OC36th / 101   
4        GPU_4       ASRock Arc B580 Challenger 12GB OC36th / 101   

  component_type Manufacturer        GPU_Chipset  Memory_Size Memory_Type  \
0            GPU        Intel           Arc B580         12.0       GDDR6   
1            GPU          AMD  Radeon RX 7700 XT         12.0       GDDR6   
2            GPU          AMD  Radeon RX 7800 XT         16.0       GDDR6   
3            GPU        Intel           Arc B580         12.0       GDDR6   
4            GPU        Intel   

In [ ]:
print("Defining compatibility functions...")

def is_compatible_cpu_mb(cpu_row, mb_row):
    """Checks CPU and Motherboard compatibility based on CPU_Socket."""
    if pd.isna(cpu_row.get('CPU_Socket')) or pd.isna(mb_row.get('CPU_Socket')):
        return False
    return cpu_row['CPU_Socket'] == mb_row['CPU_Socket']

def is_compatible_ram_mb(ram_row, mb_row):
    """Checks RAM and Motherboard compatibility based on Memory_Type."""
    if pd.isna(ram_row.get('Memory_Type')) or pd.isna(mb_row.get('Memory_Type')):
        return False
    return ram_row['Memory_Type'] == mb_row['Memory_Type']

def is_compatible_gpu_mb(gpu_row, mb_row):
    """Checks GPU and Motherboard compatibility based on PCIe x16 Slots on the motherboard."""
    if pd.isna(mb_row.get('PCIe x16 Slots')):
        return False
    # Assuming all GPUs in our dataset use PCIe x16, we only check if the MB has at least one slot.
    return mb_row['PCIe x16 Slots'] >= 1

print("Compatibility functions defined: is_compatible_cpu_mb, is_compatible_ram_mb, is_compatible_gpu_mb")


Defining compatibility functions...
Compatibility functions defined: is_compatible_cpu_mb, is_compatible_ram_mb, is_compatible_gpu_mb


## Derive Compatibility Interactions for NCF


In [ ]:
print("Filtering df_components into separate DataFrames...")
cpus_df = df_components[df_components['component_type'] == 'CPU'].copy()
motherboards_df = df_components[df_components['component_type'] == 'Motherboard'].copy()
ram_df = df_components[df_components['component_type'] == 'RAM'].copy()
gpus_df = df_components[df_components['component_type'] == 'GPU'].copy()
print("Separate DataFrames created.")

print("Generating compatibility interactions...")
interaction_data = []

# CPU and Motherboard compatibility
for idx_cpu, cpu_row in cpus_df.iterrows():
    for idx_mb, mb_row in motherboards_df.iterrows():
        compatibility = 1 if is_compatible_cpu_mb(cpu_row, mb_row) else 0
        interaction_data.append({
            'component_id_1': cpu_row['component_id'],
            'component_id_2': mb_row['component_id'],
            'compatibility': compatibility,
            'interaction_type': 'CPU_MB'
        })

# RAM and Motherboard compatibility
for idx_ram, ram_row in ram_df.iterrows():
    for idx_mb, mb_row in motherboards_df.iterrows():
        compatibility = 1 if is_compatible_ram_mb(ram_row, mb_row) else 0
        interaction_data.append({
            'component_id_1': ram_row['component_id'],
            'component_id_2': mb_row['component_id'],
            'compatibility': compatibility,
            'interaction_type': 'RAM_MB'
        })

# GPU and Motherboard compatibility
for idx_gpu, gpu_row in gpus_df.iterrows():
    for idx_mb, mb_row in motherboards_df.iterrows():
        compatibility = 1 if is_compatible_gpu_mb(gpu_row, mb_row) else 0
        interaction_data.append({
            'component_id_1': gpu_row['component_id'],
            'component_id_2': mb_row['component_id'],
            'compatibility': compatibility,
            'interaction_type': 'GPU_MB'
        })

print("Compatibility interactions generated. Converting to DataFrame...")
df_interactions = pd.DataFrame(interaction_data)

print("\n--- df_interactions Head ---")
print(df_interactions.head())

print("\n--- Compatibility Value Counts ---")
print(df_interactions['compatibility'].value_counts())

print("\n--- Interaction Type Value Counts ---")
print(df_interactions['interaction_type'].value_counts())

Filtering df_components into separate DataFrames...
Separate DataFrames created.
Generating compatibility interactions...
Compatibility interactions generated. Converting to DataFrame...

--- df_interactions Head ---
  component_id_1 component_id_2  compatibility interaction_type
0          CPU_0           MB_0              1           CPU_MB
1          CPU_0           MB_1              0           CPU_MB
2          CPU_0           MB_2              0           CPU_MB
3          CPU_0           MB_3              0           CPU_MB
4          CPU_0           MB_4              0           CPU_MB

--- Compatibility Value Counts ---
compatibility
1    939120
0    890644
Name: count, dtype: int64

--- Interaction Type Value Counts ---
interaction_type
GPU_MB    874976
CPU_MB    611892
RAM_MB    342896
Name: count, dtype: int64


## Prepare Data for NCF Model Training


In [ ]:
from sklearn.preprocessing import LabelEncoder

print("Encoding component IDs...")

# 1. Identify all unique component IDs
all_component_ids = pd.concat([
    df_interactions['component_id_1'],
    df_interactions['component_id_2']
]).unique()

# 2. Create a unique numerical ID for each unique component ID
component_id_encoder = LabelEncoder()
component_id_encoder.fit(all_component_ids)

# 3. Apply this encoding to 'component_id_1' and 'component_id_2' columns
df_interactions['component_id_1_encoded'] = component_id_encoder.transform(df_interactions['component_id_1'])
df_interactions['component_id_2_encoded'] = component_id_encoder.transform(df_interactions['component_id_2'])

print("Component IDs encoded.")
print("Total unique components after encoding:", len(component_id_encoder.classes_))

# Display the first few rows with the new encoded columns
print("\n--- df_interactions Head with Encoded IDs ---")
print(df_interactions[['component_id_1', 'component_id_1_encoded', 'component_id_2', 'component_id_2_encoded', 'compatibility']].head())

Encoding component IDs...
Component IDs encoded.
Total unique components after encoding: 3215

--- df_interactions Head with Encoded IDs ---
  component_id_1  component_id_1_encoded component_id_2  \
0          CPU_0                       0           MB_0   
1          CPU_0                       0           MB_1   
2          CPU_0                       0           MB_2   
3          CPU_0                       0           MB_3   
4          CPU_0                       0           MB_4   

   component_id_2_encoded  compatibility  
0                    2012              1  
1                    2013              0  
2                    2124              0  
3                    2235              0  
4                    2346              0  


In [ ]:
from sklearn.model_selection import train_test_split

print("Defining input features and target variable...")
X = df_interactions[['component_id_1_encoded', 'component_id_2_encoded']]
y = df_interactions['compatibility']
print("Input features (X) and target variable (y) defined.")

print("Splitting data into training, validation, and test sets...")

# First split: 80% training, 20% for temp (validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Second split: 50% of temp for validation, 50% for test (which makes 10% validation, 10% test of original data)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("Data split into training, validation, and test sets.")

print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Validation set shape: {X_val.shape}, {y_val.shape}")
print(f"Test set shape: {X_test.shape}, {y_test.shape}")

# Display class distribution for each set to verify stratification
print("\n--- Compatibility distribution in training set ---")
print(y_train.value_counts(normalize=True))
print("\n--- Compatibility distribution in validation set ---")
print(y_val.value_counts(normalize=True))
print("\n--- Compatibility distribution in test set ---")
print(y_test.value_counts(normalize=True))

Defining input features and target variable...
Input features (X) and target variable (y) defined.
Splitting data into training, validation, and test sets...
Data split into training, validation, and test sets.
Training set shape: (1463811, 2), (1463811,)
Validation set shape: (182976, 2), (182976,)
Test set shape: (182977, 2), (182977,)

--- Compatibility distribution in training set ---
compatibility
1    0.513247
0    0.486753
Name: proportion, dtype: float64

--- Compatibility distribution in validation set ---
compatibility
1    0.513248
0    0.486752
Name: proportion, dtype: float64

--- Compatibility distribution in test set ---
compatibility
1    0.513245
0    0.486755
Name: proportion, dtype: float64


## Define and Build NCF Model Architecture


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, concatenate
from tensorflow.keras.models import Model

print("Defining NCF model architecture...")

# Determine the total number of unique components for embedding layer input dimension
num_components = len(component_id_encoder.classes_)
embedding_dim = 64 # Chosen embedding dimension

# 1. Input Layers
input_component_1 = Input(shape=(1,), name='component_id_1')
input_component_2 = Input(shape=(1,), name='component_id_2')

# 2. Embedding Layers
# Embeddings for component_id_1
embedding_component_1 = Embedding(
    input_dim=num_components,
    output_dim=embedding_dim,
    input_length=1,
    name='embedding_component_1'
)(input_component_1)

# Embeddings for component_id_2
embedding_component_2 = Embedding(
    input_dim=num_components,
    output_dim=embedding_dim,
    input_length=1,
    name='embedding_component_2'
)(input_component_2)

# 3. Flatten the embedding layers' output
flatten_component_1 = Flatten(name='flatten_component_1')(embedding_component_1)
flatten_component_2 = Flatten(name='flatten_component_2')(embedding_component_2)

# 4. Concatenate the flattened embeddings
merged_embeddings = concatenate(
    [flatten_component_1, flatten_component_2],
    name='concat_embeddings'
)

# 5. Build the Multi-Layer Perceptron (MLP)
mlp_layer = Dense(256, activation='relu', name='mlp_dense_1')(merged_embeddings)
mlp_layer = Dense(128, activation='relu', name='mlp_dense_2')(mlp_layer)
mlp_layer = Dense(64, activation='relu', name='mlp_dense_3')(mlp_layer)

# 6. Output layer
output_layer = Dense(1, activation='sigmoid', name='compatibility_prediction')(mlp_layer)

# Instantiate the NCF model
ncf_model = Model(
    inputs=[input_component_1, input_component_2],
    outputs=output_layer,
    name='Neural_Collaborative_Filtering_Model'
)

print("NCF model architecture defined.")

# Print model summary
print("\n--- NCF Model Summary ---")
ncf_model.summary()

Defining NCF model architecture...
NCF model architecture defined.

--- NCF Model Summary ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "Neural_Collaborative_Filtering_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ component_id_1      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ component_id_2      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_componen… │ (None, 1, 64)     │    205,760 │ component_id_1[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_componen… │ (None, 1, 64)     │    205,760 │ component_id_2[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_component_1 │ (None, 64)        │          0 │ embedding_compon… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_component_2 │ (None, 64)        │          0 │ embedding_compon… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat_embeddings   │ (None, 128)       │          0 │ flatten_componen… │
│ (Concatenate)       │                   │            │ flatten_componen… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_1 (Dense) │ (None, 256)       │     33,024 │ concat_embedding… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_2 (Dense) │ (None, 128)       │     32,896 │ mlp_dense_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mlp_dense_3 (Dense) │ (None, 64)        │      8,256 │ mlp_dense_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ compatibility_pred… │ (None, 1)         │         65 │ mlp_dense_3[0][0] │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 485,761 (1.85 MB)

 Trainable params: 485,761 (1.85 MB)

 Non-trainable params: 0 (0.00 B)

## Train NCF Model


In [ ]:
print("Compiling the NCF model...")
ncf_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
)
print("NCF model compiled successfully.")

print("Creating EarlyStopping callback...")
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)
print("EarlyStopping callback created.")

print("Training the NCF model...")
history = ncf_model.fit(
    [X_train['component_id_1_encoded'], X_train['component_id_2_encoded']],
    y_train,
    batch_size=256,
    epochs=20,
    validation_data=(
        [X_val['component_id_1_encoded'], X_val['component_id_2_encoded']],
        y_val
    ),
    callbacks=[early_stopping_callback]
)
print("NCF model training complete.")

Compiling the NCF model...
NCF model compiled successfully.
Creating EarlyStopping callback...
EarlyStopping callback created.
Training the NCF model...
Epoch 1/20
5719/5719 ━━━━━━━━━━━━━━━━━━━━ 60s 10ms/step - accuracy: 0.9709 - auc: 0.9919 - loss: 0.0632 - precision: 0.9740 - recall: 0.9721 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 5.2774e-05 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/20
5719/5719 ━━━━━━━━━━━━━━━━━━━━ 58s 10ms/step - accuracy: 0.9999 - auc: 1.0000 - loss: 2.4805e-04 - precision: 1.0000 - recall: 0.9999 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 7.3454e-05 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/20
5719/5719 ━━━━━━━━━━━━━━━━━━━━ 56s 10ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 1.4067e-04 - precision: 1.0000 - recall: 0.9999 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 5.1411e-05 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/20
5719/5719 ━━━━━━━━━━━━━━━━━━━━ 57s 10ms/step - accuracy: 1.0000 - auc: 1.0000 

In [ ]:
print("Evaluating the NCF model on the test set...")
results = ncf_model.evaluate(
    [X_test['component_id_1_encoded'], X_test['component_id_2_encoded']],
    y_test
)

print("NCF Model Evaluation Results:")
print(f"  Loss: {results[0]:.4f}")
print(f"  Accuracy: {results[1]:.4f}")
print(f"  Precision: {results[2]:.4f}")
print(f"  Recall: {results[3]:.4f}")
print(f"  AUC: {results[4]:.4f}")

Evaluating the NCF model on the test set...
5719/5719 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 1.5129e-04 - precision: 1.0000 - recall: 1.0000
NCF Model Evaluation Results:
  Loss: 0.0002
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  AUC: 1.0000


In [ ]:
import pickle

print("Saving the trained NCF model...")
ncf_model.save('ncf_model.h5')
print("NCF model saved as 'ncf_model.h5'.")

print("Saving the component ID encoder...")
with open('component_id_encoder.pkl', 'wb') as f:
    pickle.dump(component_id_encoder, f)
print("Component ID encoder saved as 'component_id_encoder.pkl'.")

Saving the trained NCF model...
NCF model saved as 'ncf_model.h5'.
Saving the component ID encoder...
Component ID encoder saved as 'component_id_encoder.pkl'.


In [ ]:
import pickle

print("Saving the trained NCF model...")
ncf_model.save('ncf_model.keras')
print("NCF model saved as 'ncf_model.keras'.")

print("Saving the component ID encoder...")
with open('component_id_encoder.pkl', 'wb') as f:
    pickle.dump(component_id_encoder, f)
print("Component ID encoder saved as 'component_id_encoder.pkl'.")

Saving the trained NCF model...
NCF model saved as 'ncf_model.keras'.
Saving the component ID encoder...
Component ID encoder saved as 'component_id_encoder.pkl'.
